In [2]:
import sys, os

from sqlalchemy.sql.expression import table

sys.path.append(
    os.path.dirname(
        os.path.abspath(
            ""
        )
    )
)



from models import Statements_list_table
from models import Security, Earnings_release
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from scrapping_sources.Macrotrend import Macrotrend
import datetime
import pandas as pd

USERNAME='postgres'
HOST='database-webscrap.ccaaerr6cq44.eu-west-3.rds.amazonaws.com'
DATABASE_NAME='webscrapping'
PASSWORD='postgres5678'
PORT=5432
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

engine = create_engine(DATABASE_URI)
session = sessionmaker(bind= engine)
s = session()
M = Macrotrend()

s = session()
r = s.query(Security.ticker, Security.id).all()
sec_id_map = {i[0]:i[1] for i in r}

statements = [
    'income-statement',
    'balance-sheet',
    'cash-flow-statement',
    'financial-ratios'
]
time_format = [
    'annual',
    'quarterly'
]

def get_statement_list__DB__(ticker):
    """Returns a dataframe containing all statements of a particular ticker"""
    r = s.query(
        Statements_list_table.date,
        Statements_list_table.ticker,
        Statements_list_table.statement,
        Statements_list_table.security_id,
        # Statements_list_table.statement_id
    ).where(
        Statements_list_table.ticker == ticker,
    ).all()

    df = pd.DataFrame(r)
    df.columns = [['date','ticker','statement','security_id']]

    return df

def get_statement_list__M__(ticker, statement, time_format):
    """Generates a statements list from macrotrend based on the arguments:
        -ticker,
        -statement,
        -time_format"""
    df = M.arrange_data(ticker, statement, time_format)
    df = df[['date','ticker', 'statement', 'security_id']].drop_duplicates()
    df["security_id"] = df["ticker"].map(sec_id_map)

    return df


def get_all_statements__M__(ticker):
    """generates a statement list from Macrotrend with all statements
    and time formats combined, for a ticker"""
    table = []

    for stmnt in statements:
        for t_format in time_format:

            df = get_statement_list__M__(ticker, stmnt, t_format)
            table.append(df)

    results = pd.concat(table)
    results.columns = [['date','ticker','statement','security_id']]

    return results

def generate_list_update_ticker(ticker):

    db = get_statement_list__DB__(ticker)
    m = get_all_statements__M__(ticker)
    update = pd.concat([db,m]).drop_duplicates(keep=False)
    return update


In [5]:

DAY_VARIABLE = 30
look_back_date = str(datetime.date.today() - datetime.timedelta(days=DAY_VARIABLE))

r = s.query(Earnings_release.__table__).filter(Earnings_release.release_date >= look_back_date).all()
earnings_df = pd.DataFrame(r)
earnings_df.columns = Earnings_release.__table__.columns.keys()

# 1 - We are not concerned with tickers that are not in our DB
df = earnings_df[earnings_df['last_period_DB'].notna()]
df = df[df['last_period_DB'] != 'None']

# 2 - Remove all the rows that dont have data on Trend
df = df[df['last_period_M'].notna()]

# 3 - Remove all the rows where DB = N
df = df[df['last_period_DB'] != df.last_period_N]

# 4 - Remove all the rows where db == m
# df = df[df['last_period_DB'] != df['last_period_M']]
df


for row in df.iterrows():

    id_ = row[1][0]
    ticker = row[1][3]
    # LATEST PERIOD M THAT WE HAVE ON OUR DATABASE
    last_period_M_on_record = row[1][7]
    on_DB = row[1][6]
    M_latest = M.latest_ending_period_available(ticker)

    if last_period_M_on_record != M_latest:
        try:

            # UPDATE STATEMENTS_LIST
            generate_list_update_ticker(ticker)
            # UPDATE STATEMENTS_DETAIL
            # TODO: get the code from update_statements_v2.py
            # UPDATE EARNINGS TABLE
            s.query(
                Earnings_release.__table__
            ).filter(
                Earnings_release.id == id_
            ).update(
                {
                    "last_period_M": f"{M_latest}"
                }
            )
            s.commit()
        except:
            pass


# I/ Check what earnings needs to be update thanks to earnings_release
#   |-- Apply the same logic used in update-statement script
#   |-- Filter out tickers that are not in db
#   |-- Filter out 
# II/ Create a list of tickers that need to be updated

STZ <Response [200]>
RPM <Response [200]>
LEVI <Response [200]>
AYI <Response [200]>
IDT <Response [200]>
RGP <Response [200]>
LOOP <Response [200]>
VLGEA <Response [200]>
STCN <Response [200]>
RELL <Response [200]>
EDUC <Response [200]>
HIHO <Response [200]>
PEP <Response [200]>
SAR <Response [200]>
KOR <Response [200]>
LNDC <Response [200]>
VWE <Response [200]>
NKE <Response [200]>
COST <Response [200]>
TCOM <Response [200]>
PRGS <Response [200]>
AEHR <Response [200]>


In [6]:
df

,id,date,release_date,ticker,security_id,last_period_N,last_period_DB,last_period_M
0,13,2021-10-07,2021-10-06,STZ,7079.0,2021-08,2021-05,2021-05
2,15,2021-10-07,2021-10-06,RPM,6399.0,2021-08,2021-05,2021-05
3,16,2021-10-07,2021-10-06,LEVI,4454.0,2021-08,2021-05,2021-05
4,17,2021-10-07,2021-10-06,AYI,660.0,2021-08,2021-05,2021-05
5,18,2021-10-07,2021-10-06,IDT,3722.0,2021-07,2021-04,2021-04
6,19,2021-10-07,2021-10-06,RGP,6297.0,2021-08,2021-05,2021-05
7,20,2021-10-07,2021-10-06,LOOP,4550.0,2021-08,2021-05,2021-05
8,21,2021-10-07,2021-10-06,VLGEA,7831.0,2021-07,2021-04,2021-04
9,22,2021-10-07,2021-10-06,STCN,7032.0,2021-07,2021-04,2021-04
10,23,2021-10-07,2021-10-06,RELL,6250.0,2021-08,2021-05,2021-05


In [19]:


for ticker in todays_updated_tickers:

    db =  get_statement_list_db(ticker)
    m = generate_statement_list_all(ticker)
    update= pd.concat([m,db]).drop_duplicates(keep=False)
    update.columns = ['date', 'ticker', 'statement', 'security_id']
    print("UPDATE :", update)

    if not update.empty:

        try:
            update.to_sql(con = engine, name="statements_list_table", if_exists='append', index=False)
        except:
            print("Up to date")


TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
UPDATE :            date ticker statement  security_id
0    2020-11-30    TMQ      IS-A         7348
21   2019-11-30    TMQ      IS-A         7348
42   2018-11-30    TMQ      IS-A         7348
63   2017-11-30    TMQ      IS-A         7348
84   2016-11-30    TMQ      IS-A         7348
..          ...    ...       ...          ...
215  2011-11-30    TMQ      BS-Q         7348
216  2011-08-31    TMQ      BS-Q         7348
217  2011-05-31    TMQ      BS-Q         7348
218  2011-02-28    TMQ      BS-Q         7348
219  2010-11-30    TMQ      BS-Q         7348

[440 rows x 4 columns]
Up to date


In [17]:
update.columns = ['date','ticker','statement','security_id']
update.unstack()
update.columns

Index(['date', 'ticker', 'statement', 'security_id'], dtype='object')

In [24]:
generate_statement_list_all('TMQ')

TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>
TMQ <Response [200]>


,date,ticker,statement,security_id
0,2020-11-30,TMQ,IS-A,7348
21,2019-11-30,TMQ,IS-A,7348
42,2018-11-30,TMQ,IS-A,7348
63,2017-11-30,TMQ,IS-A,7348
84,2016-11-30,TMQ,IS-A,7348
...,...,...,...,...
741,2011-11-30,TMQ,R-Q,7348
760,2011-08-31,TMQ,R-Q,7348
779,2011-05-31,TMQ,R-Q,7348
798,2011-02-28,TMQ,R-Q,7348


In [27]:
get_todays_statements_update()

,0,1,2,3
0,TMQ,annual,income-statement,2021-10-09
1,TMQ,quarterly,income-statement,2021-10-09
2,TMQ,annual,balance-sheet,2021-10-09
3,TMQ,quarterly,balance-sheet,2021-10-09
4,TMQ,annual,cash-flow-statement,2021-10-09
5,TMQ,quarterly,cash-flow-statement,2021-10-09
6,TMQ,annual,financial-ratios,2021-10-09
7,TMQ,quarterly,financial-ratios,2021-10-09


In [3]:

r = s.query(
Statements_list_table.ticker,
Statements_list_table.date ,
Statements_list_table.statement,
Statements_list_table.statement_id
).all()

r

[('A', datetime.date(2020, 10, 31), 'IS-A', 226818),
 ('A', datetime.date(2019, 10, 31), 'IS-A', 226819),
 ('A', datetime.date(2018, 10, 31), 'IS-A', 226820),
 ('A', datetime.date(2017, 10, 31), 'IS-A', 226821),
 ('A', datetime.date(2016, 10, 31), 'IS-A', 226822),
 ('A', datetime.date(2015, 10, 31), 'IS-A', 226823),
 ('A', datetime.date(2014, 10, 31), 'IS-A', 226824),
 ('A', datetime.date(2013, 10, 31), 'IS-A', 226825),
 ('A', datetime.date(2012, 10, 31), 'IS-A', 226826),
 ('A', datetime.date(2011, 10, 31), 'IS-A', 226827),
 ('A', datetime.date(2010, 10, 31), 'IS-A', 226828),
 ('A', datetime.date(2009, 10, 31), 'IS-A', 226829),
 ('A', datetime.date(2008, 10, 31), 'IS-A', 226830),
 ('A', datetime.date(2007, 10, 31), 'IS-A', 226831),
 ('A', datetime.date(2006, 10, 31), 'IS-A', 226832),
 ('A', datetime.date(2005, 10, 31), 'IS-A', 226833),
 ('AAC', datetime.date(2020, 12, 31), 'IS-A', 226834),
 ('AACG', datetime.date(2020, 12, 31), 'IS-A', 226835),
 ('AACG', datetime.date(2019, 12, 31), 'I

In [59]:
for i in r

In [45]:
udf = pd.DataFrame(r)

In [47]:
udf.columns = [['id', 'date', 'statement', 'ticker', 'security_id', 'line_item','amount']]

In [60]:
# # df['d'] = df.apply(lambda x: retrieve_statement_id(date = x['date'], ticker= x['ticker']), axis=1)
# udf['test'] = udf.apply(lambda x: statement_list_mapping(ticker = x['ticker'], date= x['date'], statement= x['statement']), axis=1)

In [61]:
s.close_all()

/tmp/ipykernel_142169/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()
